<a href="https://colab.research.google.com/github/sujunghwang/codestatesproject/blob/main/CP2-1_%EB%89%B4%EC%8A%A4_%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4jzcwsm6
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4jzcwsm6
     |████████████████████████████████| 132 kB 5.5 MB/s 
     |████████████████████████████████| 344 kB 34.7 MB/s 
     |████████████████████████████████| 47.3 MB 76 kB/s 
     |████████████████████████████████| 4.5 MB 36.2 MB/s 
     |████████████████████████████████| 1.2 MB 13.0 MB/s 
     |████████████████████████████████| 3.8 MB 32.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
     |████████████████████████████████| 6.5 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 36.9 MB/s 
     |████████████████████████████████| 8.6 MB 36.6 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 127 kB 44.5 MB/s 
  Created wheel for kobert: filename=kobe

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

import os


from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
device = torch.device("cuda:0")                                                                # GPU 사용

bertmodel, vocab = get_pytorch_kobert_model()                                                  # BERT 모델, Vocabulary 불러오기

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [4]:
category = []                                                                                  # 뉴스 카테고리를 담을 리스트
root = "drive/MyDrive/cp2/"

for cat in os.listdir(root):
    files = os.listdir(root + cat)
    category.append(cat)
  
category

['ITscience',
 'economy',
 'politic',
 'social',
 'sport',
 'entertainment',
 'life',
 'culture',
 'health']

In [5]:
dataset_train = []
dataset_test = []

list = os.listdir(root)
for cat in list:
    files = os.listdir(root + cat)
    for i,f in enumerate(files):
        fname = root + cat + "/" + f
        file = open(fname, "r")
        strings = file.read()
        if i<1200:                                                                             # 각 카테고리 당 1200개를 train set, 200개를 test set으로 사용
            dataset_train.append([strings, category.index(cat)])                               # label은 category.index()를 통해 숫자로 넣기
        else:
            dataset_test.append([strings,category.index(cat)])
        file.close()

print(len(dataset_train), len(dataset_test))

10800 1800


In [6]:
dataset_train[0]                                                                               # [문장, 카테고리 인덱스]

['기타 신동도 이 대통령도 유튜브로 돈 벌 수 있다\n한국 개인 사용자들이 전 세계를 상대로 수익을 얻을 기회다.\n온라인 동영상 사이트 유튜브는 6일 오전 10시 30분 서울 역삼동 구글코리아 본사에서 유튜브 3.0을 소개하는 기자간담회를 열었다.\n유튜브는 하루 조회수 30억 건에 이르는 세계 최대 온라인 동영상 사이트로 한국을 포함해 전세계 32개국에서 서비스하고 있다.\n구글 검색, 지메일 등 다른 서비스들이 네이버, 다음과 같은 토종 포털에 밀려 한국 시장에서 고전하고 있는 와중에도 유튜브는 국내 동영상 사이트 1위를 고수해 왔다.\n유튜브는 이날 2008년 1월 한국 서비스 당시 단순히 사용자들이 동영상을 올리고 나누는 개념이었던 1.0세대, SM, JYP, YG 등 국내 파트너사들과 수익을 배분하는 2.0 세대에 이어 개인 사용자들과도 광고 수익을 나누는 3.0 세대 개념을 발표했다.\n일본 도쿄에서 화상으로 연결된 아담 스미스 유튜브 아시아태평양지역 총괄은 유튜브는 이제 스마트폰, 태블릿, TV 등을 통해 언제, 어디서, 어떤 기기로든 시청할 수 있다면서 창의적 사용자들이 동영상에 광고를 붙여 수익화할 수 있게 유튜브가 도와줄 것이라고 밝혔다.\n스미스 총괄은 기타 연주 동영상을 유튜브에 올려 3억 회 이상 조회수를 기록한 14살 기타 신동 정성하군 사례를 들며 파트너를 일반 사용자까지 확대하겠다고 밝혔다.\n그동안 동영상 저작권 침해 문제로 콘텐츠 사업자들과 크고 작은 마찰을 빚었던 유튜브는 저작권 보호 솔루션인 콘텐츠 ID 기술을 도입해 세계 미디어 기업과 파트너 관계를 맺고 있다.\n콘텐츠 ID는 파트너사에서 동영상을 올린 뒤 일반 사용자들이 같은 동영상을 올릴 경우 이를 가려내 파트너 스스로 저작권을 관리할 수 있게 하는 기술이다.\n스미스는 콘텐츠ID 덕분에 파트너사 저작권을 보호하고 수익 창출이 가능해 세계 굴지 미디어 기업과 파트너를 맺을 수 있다, 아시아에선 한국 K팝과 한류가 성공 사례로 꼽힌다면서 SM, YG, JYP 등 한국

In [7]:
tokenizer = get_tokenizer()                                                                    # 가져온 vocab을 이용하여 토크나이저를 선언
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
# BERTSentenceTransform 를 이용해 토큰화와 패딩 진행
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [10]:
#Dataset 선언, Dataloader를 생성
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,                                                                # 뉴스 카테고리가 9개 이므로 9로 설정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out) 

In [12]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()


t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [13]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [14]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/169 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 train acc 0.49352810650887574


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.7834051724137931


  0%|          | 0/169 [00:00<?, ?it/s]

epoch 10 train acc 0.992603550295858


  0%|          | 0/29 [00:00<?, ?it/s]

epoch 10 test acc 0.7823275862068966


In [23]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("뉴스 텍스트: ", seq)
    print("신뢰도:", "{:.2f}%".format(softmax(result,idx)))
    print("뉴스의 카테고리:", category[idx])
    return softmax(result,idx)

In [21]:
test_list = [
    ["구글·애플, 앱 마켓 외부결제 못 막는다…전기통신사업법 시행령 15일 개정", "ITscience"],
    ["오스카 작품상도 애플 품에… 할리우드 'OTT 전성시대 인증'", "entertianment"],
    ["산업부, 366억 투입해 대전 지역 사회적경제 기업 지원 거점 마련", "economy"],
    ["민주 원내 비서실장에 전용기…선임부대표에 김정호·신정훈", "politic"],
    ["도로 한복판서 오토바이 배달기사 무차별 폭행…경찰 조사", "social"],
    ["‘컴퓨터 세터’ 과외, 고속 토스 이끌다", "sport"],
    ["예후 나쁜 간암은 유전자변이 많고 주변 혈관 침습 흔해", "life"],
    ["'Seen and Unseen' 메리 코스 개인전", "culture"],
    ["쌍용차 재매각마저 무산..산업은행 또 자금지원 나설까", "economy"],
    ["'679홈런 전설' 푸홀스, 친정팀 STL과 1년 250만$ 계약 합의", "sport"],
    ["임영웅→아이유 모창 능력자 모집 '히든싱어7', 올 여름 방송", "entertianment"]
]

In [27]:
acc = []
for i in test_list:
  testModel(model, i[0])
  acc.append(testModel(model, i[0]))
  print('실제 카테고리: ', i[1])
  print()
  
print('평균 신뢰도: ', (sum(acc) / len(acc)))

뉴스 텍스트:  구글·애플, 앱 마켓 외부결제 못 막는다…전기통신사업법 시행령 15일 개정
신뢰도: 99.74%
뉴스의 카테고리: ITscience
뉴스 텍스트:  구글·애플, 앱 마켓 외부결제 못 막는다…전기통신사업법 시행령 15일 개정
신뢰도: 99.74%
뉴스의 카테고리: ITscience
실제 카테고리:  ITscience

뉴스 텍스트:  오스카 작품상도 애플 품에… 할리우드 'OTT 전성시대 인증'
신뢰도: 99.69%
뉴스의 카테고리: ITscience
뉴스 텍스트:  오스카 작품상도 애플 품에… 할리우드 'OTT 전성시대 인증'
신뢰도: 99.69%
뉴스의 카테고리: ITscience
실제 카테고리:  entertianment

뉴스 텍스트:  산업부, 366억 투입해 대전 지역 사회적경제 기업 지원 거점 마련
신뢰도: 90.12%
뉴스의 카테고리: economy
뉴스 텍스트:  산업부, 366억 투입해 대전 지역 사회적경제 기업 지원 거점 마련
신뢰도: 90.12%
뉴스의 카테고리: economy
실제 카테고리:  economy

뉴스 텍스트:  민주 원내 비서실장에 전용기…선임부대표에 김정호·신정훈
신뢰도: 99.62%
뉴스의 카테고리: politic
뉴스 텍스트:  민주 원내 비서실장에 전용기…선임부대표에 김정호·신정훈
신뢰도: 99.62%
뉴스의 카테고리: politic
실제 카테고리:  politic

뉴스 텍스트:  도로 한복판서 오토바이 배달기사 무차별 폭행…경찰 조사
신뢰도: 99.11%
뉴스의 카테고리: social
뉴스 텍스트:  도로 한복판서 오토바이 배달기사 무차별 폭행…경찰 조사
신뢰도: 99.11%
뉴스의 카테고리: social
실제 카테고리:  social

뉴스 텍스트:  ‘컴퓨터 세터’ 과외, 고속 토스 이끌다
신뢰도: 99.57%
뉴스의 카테고리: ITscience
뉴스 텍스트:  ‘컴퓨터 세터’ 과외, 고속 토스 이끌다
신뢰도: 99.57%
뉴스의 카테고리: ITscience
실제 카테고리:  sport

뉴스 